In [37]:
import cv2
import numpy as np
import os

In [ ]:
import cv2
import numpy as np
import os

# --- Global Configuration and State ---
# Maximum dimension (width or height) the image should have on screen
MAX_DISPLAY_DIM = 900
# Name of the window where the image is displayed
WINDOW_NAME = 'Click the 4 Corners (Source) - Press Q to Quit'

# Transformation target size (Destination Points)
# Updated to match a 15cm x 12cm ratio (5:4) for better geometric accuracy.
DEST_WIDTH = 600 
DEST_HEIGHT = 480 
destination_points = np.float32([
    [0, 0],                            # Target Top-Left
    [DEST_WIDTH - 1, 0],               # Target Top-Right
    [DEST_WIDTH - 1, DEST_HEIGHT - 1], # Target Bottom-Right
    [0, DEST_HEIGHT - 1]               # Target Bottom-Left
])

# Global variables
source_points_scaled = [] # Coordinates clicked on the smaller, displayed image
scale_factor = 1.0        # Factor by which the original image was scaled down
original_img = None       # The full-resolution image
display_img = None        # The scaled-down image used for display

def order_points(pts):
    """
    Initializes a list of coordinates that will be ordered such that the first
    entry in the list is the top-left, the second is the top-right, the third
    is the bottom-right, and the fourth is the bottom-left.

    Uses sum and difference of coordinates for robust corner detection.
    
    Returns: A NumPy array of shape (4, 2) in the order: TL, TR, BR, BL.
    """
    pts = np.array(pts, dtype="float32")
    rect = np.zeros((4, 2), dtype="float32")

    # 1. Identify Top-Left (TL) and Bottom-Right (BR) based on the sum (x + y).
    s = pts.sum(axis=1)
    tl_idx = np.argmin(s)
    br_idx = np.argmax(s)
    rect[0] = pts[tl_idx] # Top-Left
    rect[2] = pts[br_idx] # Bottom-Right

    # 2. Identify Top-Right (TR) and Bottom-Left (BL) from the remaining two points.
    remaining_indices = [i for i in range(4) if i != tl_idx and i != br_idx]
    unassigned_pts = pts[remaining_indices]
    
    # Calculate the difference (x - y) for the two unassigned points
    diff_0 = unassigned_pts[0, 0] - unassigned_pts[0, 1]
    diff_1 = unassigned_pts[1, 0] - unassigned_pts[1, 1]

    # The point with the LARGER difference is Top-Right (TR), the smaller is Bottom-Left (BL).
    if diff_0 > diff_1:
        rect[1] = unassigned_pts[0] # Top-Right (TR)
        rect[3] = unassigned_pts[1] # Bottom-Left (BL)
    else:
        rect[1] = unassigned_pts[1] # Top-Right (TR)
        rect[3] = unassigned_pts[0] # Bottom-Left (BL)

    return rect

def process_with_hough_circles(transformed_img):
    """
    Processes the image to segment coins using the Hough Circle Transform, 
    which is robust to broken edges and specifically designed for circular objects.
    """
    # 1. Convert to grayscale and apply Gaussian blur 
    gray = cv2.cvtColor(transformed_img, cv2.COLOR_BGR2GRAY)
    
    # Aggressive blur to smooth internal coin details and improve circle detection
    blurred = cv2.GaussianBlur(gray, (9, 9), 2) 

    # 2. Apply Hough Circle Transform
    # dp=1: Accumulator resolution. Full resolution.
    # minDist=20: Minimum distance between centers (prevents multiple circles on one coin).
    # param1=50: Higher threshold for Canny edge detection (passed to Canny internally).
    # param2=30: Accumulator threshold for circle centers (lower means more circles detected).
    # minRadius=20, maxRadius=50: Expected size range for the coins (adjust if needed).
    circles = cv2.HoughCircles(
        blurred, 
        cv2.HOUGH_GRADIENT, 
        dp=1, 
        minDist=20,
        param1=50, 
        param2=30, 
        minRadius=20, 
        maxRadius=50
    )
    
    # 3. Create a blank black image for the binary mask
    clean_binary_img = np.zeros_like(gray)

    if circles is not None:
        # Convert the (x, y) coordinates and radius (r) to integers
        circles = np.uint16(np.around(circles))
        
        # 4. Draw the detected circles onto the binary mask
        for i in circles[0, :]:
            center = (i[0], i[1])
            radius = i[2]
            # Draw the detected circle (coin) filled with white
            cv2.circle(clean_binary_img, center, radius, 255, thickness=cv2.FILLED)
    
    # 5. Display and Save the result
    cv2.imshow('Binary Coin Mask (Hough Circles)', clean_binary_img)
    
    output_filename = 'transformed_binary_output_hough.png'
    cv2.imwrite(output_filename, clean_binary_img)
    print(f"Binary image saved as '{output_filename}'")
    
    return clean_binary_img


def apply_perspective_transform():
    """
    Calculates the Homography matrix and applies the perspective transform
    using the full-resolution image and upscaled points.
    """
    global original_img, source_points_scaled, scale_factor

    if len(source_points_scaled) != 4:
        print("Error: Exactly 4 points must be selected.")
        return

    # 1. Sort the selected points into the correct order (TL, TR, BR, BL)
    sorted_scaled_points = order_points(source_points_scaled)
    
    # 2. SCALE UP the clicked points to match the original image size
    source_points_original = sorted_scaled_points / scale_factor
    
    # 3. Calculate the Homography Matrix (M)
    matrix = cv2.getPerspectiveTransform(source_points_original, destination_points)

    # 4. Apply the Perspective Transformation to the ORIGINAL image
    transformed_img = cv2.warpPerspective(
        original_img, 
        matrix, 
        (DEST_WIDTH, DEST_HEIGHT),
        # Use bilinear interpolation for better quality when scaling
        flags=cv2.INTER_LINEAR 
    )

    # 5. Display the color result
    cv2.imshow('Transformed Rectangle (Destination)', transformed_img)
    print("\n--- Transformation Complete ---")
    print(f"Transformed color image displayed ({DEST_WIDTH}x{DEST_HEIGHT}).")
    
    # 6. Process the transformed image to create a binary mask using Hough Circles
    process_with_hough_circles(transformed_img)

    # Reset the points list for a new selection if needed
    source_points_scaled = []
    # Refresh the drawing on the display image
    refresh_display_image()
    cv2.imshow(WINDOW_NAME, display_img)


def refresh_display_image():
    """Creates a fresh copy of the display image and draws the selected points."""
    global display_img, original_img, scale_factor
    
    # Recalculate the displayed image in case the original was modified (though it shouldn't be here)
    h, w = original_img.shape[:2]
    new_w = int(w * scale_factor)
    new_h = int(h * scale_factor)
    display_img = cv2.resize(original_img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Draw the selected points on the display image
    for i, pt in enumerate(source_points_scaled):
        # Draw a circle for the point
        cv2.circle(display_img, tuple(pt), 5, (0, 255, 0), -1) # Green filled circle
        
        # Label the point
        label = str(i + 1)
        cv2.putText(display_img, label, (pt[0] + 10, pt[1] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2) # Yellow label

    # Draw lines connecting the selected points
    if len(source_points_scaled) > 1:
        for i in range(len(source_points_scaled) - 1):
            cv2.line(display_img, tuple(source_points_scaled[i]), tuple(source_points_scaled[i+1]), (255, 0, 0), 2) # Blue line
        
        # Close the rectangle when 4 points are selected
        if len(source_points_scaled) == 4:
            cv2.line(display_img, tuple(source_points_scaled[3]), tuple(source_points_scaled[0]), (255, 0, 0), 2)


# --- Mouse Callback Function ---
def mouse_callback(event, x, y, flags, param):
    """Handles mouse clicks to select the 4 corner points on the scaled image."""
    global source_points_scaled, display_img

    if event == cv2.EVENT_LBUTTONDOWN:
        if len(source_points_scaled) < 4:
            # Add the clicked point (which are scaled coordinates)
            source_points_scaled.append([x, y])
            print(f"Point {len(source_points_scaled)} selected at: ({x}, {y}) [Scaled]")

            # Redraw the image with the new point
            refresh_display_image()
            cv2.imshow(WINDOW_NAME, display_img)

            # If 4 points are collected, trigger the transform
            if len(source_points_scaled) == 4:
                apply_perspective_transform()
        else:
            print("Already selected 4 points. Press 'C' to clear and select again.")

# --- Main Execution ---
def main():
    """Initializes the application, handles scaling, and starts the main loop."""
    global original_img, display_img, scale_factor
    
    input_file = "coins3.jpg"

    if not os.path.exists(input_file):
        print(f"Error: Image file '{input_file}' not found.")
        print("Please ensure the image is in the same directory as the script.")
        return

    original_img = cv2.imread(input_file)
    if original_img is None:
        print(f"Error: Could not read image file '{input_file}'. Check file permissions or format.")
        return

    # --- Automatic Scaling Logic ---
    h, w = original_img.shape[:2]
    
    # Calculate scale factor if the image exceeds the maximum display size
    if w > MAX_DISPLAY_DIM or h > MAX_DISPLAY_DIM:
        scale_w = MAX_DISPLAY_DIM / w
        scale_h = MAX_DISPLAY_DIM / h
        # Use the smaller scale factor to ensure both dimensions fit
        scale_factor = min(scale_w, scale_h)
        
        print(f"Original size: {w}x{h}. Scaling down by factor: {scale_factor:.2f}")
    else:
        scale_factor = 1.0
        print(f"Image size {w}x{h} fits screen. No scaling applied.")

    # Create the initial display image (already handled in refresh_display_image, but done here for clarity)
    new_w = int(w * scale_factor)
    new_h = int(h * scale_factor)
    display_img = cv2.resize(original_img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # 3. Setup the OpenCV window and callback
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL) # Use WINDOW_NORMAL to allow resizing if needed
    cv2.setMouseCallback(WINDOW_NAME, mouse_callback)

    print("\n--- Instructions ---")
    print(f"1. Click the four corners of the rectangle you want to flatten in {input_file}.")
    print("   The order in which you click the points does not matter; they will be sorted automatically.")
    print("2. After 4 clicks, the transformed color image and the robustly segmented binary mask will appear.")
    print("3. The binary mask uses the Hough Circle Transform and is automatically saved as 'transformed_binary_output_hough.png'.")
    print("4. Press 'Q' to quit all windows.")
    print("5. Press 'C' to clear selected points and start a new selection.")
    
    # 4. Main loop to keep the window open
    while True:
        cv2.imshow(WINDOW_NAME, display_img)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            # Quit on 'q' press
            break
        elif key == ord('c'):
            # Clear points on 'c' press
            source_points_scaled = []
            refresh_display_image()
            cv2.imshow(WINDOW_NAME, display_img)
            # Try to destroy the transform windows if they exist
            try:
                cv2.destroyWindow('Transformed Rectangle (Destination)')
                cv2.destroyWindow('Binary Coin Mask (Hough Circles)')
            except cv2.error:
                pass # Window probably hasn't been created yet
            print("Selected points cleared. Start clicking again.")

    # 5. Cleanup
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Original size: 1496x1446. Scaling down by factor: 0.60

--- Instructions ---
1. Click the four corners of the rectangle you want to flatten in coins3.jpg.
   The order in which you click the points does not matter; they will be sorted automatically.
2. After 4 clicks, the transformed color image and the robustly segmented binary mask will appear.
3. The binary mask uses the Hough Circle Transform and is automatically saved as 'transformed_binary_output_hough.png'.
4. Press 'Q' to quit all windows.
5. Press 'C' to clear selected points and start a new selection.
Point 1 selected at: (18, 400) [Scaled]
Point 2 selected at: (377, 28) [Scaled]
Point 3 selected at: (858, 403) [Scaled]
Point 4 selected at: (548, 849) [Scaled]

--- Transformation Complete ---
Transformed color image displayed (600x480).
Binary image saved as 'transformed_binary_output_hough.png'
Point 1 selected at: (19, 408) [Scaled]
Point 2 selected at: (374, 25) [Scaled]
Point 3 selected at: (854, 398) [Scaled]
Point 4 sel